## Purpose and Objective

The purpose of this data mining project is to study an issue that affects healthcare economics. The objective is to understand the dataset through EDA and use various machine learning models to predic payment trends.

In [1]:
#import libraries
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# for Mac errors
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.ticker as ticker
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## Data Preprocessing

Clean and prepare data for data analysis


    1. Import Dataset
    2. Remove punctuations causing errors
    3. Convert attributes to proper data types
    4. Handle null values
    5. Handle payment column
    6. Establish metrics
    7. standardize numeric variables
    8. identify outliers

#### 1. Import Dataset

In [2]:
# load data
full_cbsa = pd.read_csv("cbsa2.csv")
full_cbsa.head(5)

,reference_period,type_of_service,aggregation_level,cbsa,cbsatitle,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,...,average_number_of_providers_per_cbsa_dual_color,average_number_of_providers_per_cbsa_description,number_of_dual_eligible_users_dual_color,number_of_dual_eligible_users_description,percentage_of_dual_eligible_users_out_of_total_users_dual_color,percentage_of_dual_eligible_users_out_of_total_users_description,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries_dual_color,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries_dual_color_description,total_payment_dual_color,total_payment_description
0,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),NATION + TERRITORIES,--ALL--,--ALL--,"37,359,009","9,078",424.34,10.31%,"3,852,199",...,,,,,,,,,,
1,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10100,"Aberdeen, SD","7,526",3,231,9.21%,693,...,BLUE 1,Lowest 25% (Less than 5.00 Providers),BLUE 2,Second Lowest 25% (166 - 349 Dual Eligible Users),BLUE 2,Second Lowest 25% (21.67% - 26.95% of Total Us...,BLUE 3,Third Lowest 25% (17.06% - 19.32% of Total FFS...,BLUE 2,"Second Lowest 25% ($525,929.51 - $1,042,852.56 )"
2,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10140,"Aberdeen, WA","17,349",18,110.44,11.46%,"1,988",...,BLUE 4,Top 25% Excl. Extreme Values (16.00 - 32.49 Pr...,BLUE 3,Third Lowest 25% (350 - 845 Dual Eligible Users),BLUE 4,Top 25% Excl. Extreme Values (33.58% - 51.44% ...,BLUE 4,Top 25% Excl. Extreme Values (19.33% - 27.78% ...,BLUE 3,"Third Lowest 25% ($1,042,852.57 - $2,686,138.28 )"
3,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10180,"Abilene, TX","25,198",11,271.09,11.83%,"2,982",...,BLUE 3,Third Lowest 25% (9.00 - 15.99 Providers),BLUE 4,"Top 25% Excl. Extreme Values (846 - 1,864 Dual...",BLUE 3,Third Lowest 25% (26.96% - 33.57% of Total Users),BLUE 4,Top 25% Excl. Extreme Values (19.33% - 27.78% ...,BLUE 3,"Third Lowest 25% ($1,042,852.57 - $2,686,138.28 )"
4,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10220,"Ada, OK","7,570",5,104.8,6.92%,524,...,BLUE 2,Second Lowest 25% (5.00 - 8.99 Providers),BLUE 2,Second Lowest 25% (166 - 349 Dual Eligible Users),BLUE 4,Top 25% Excl. Extreme Values (33.58% - 51.44% ...,BLUE 1,Lowest 25% (Less than 13.69% of Total FFS Bene...,BLUE 1,"Lowest 25% (Less than $525,929.51 )"


In [3]:
full_cbsa.shape

(163035, 35)

In [4]:
cbsa = full_cbsa.iloc[:, :15]
cbsa.head()

,reference_period,type_of_service,aggregation_level,cbsa,cbsatitle,number_of_fee_for_service_beneficiaries,number_of_providers,average_number_of_users_per_provider,percentage_of_users_out_of_ffs_beneficiaries,number_of_users,average_number_of_providers_per_cbsa,number_of_dual_eligible_users,percentage_of_dual_eligible_users_out_of_total_users,percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,total_payment
0,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),NATION + TERRITORIES,--ALL--,--ALL--,"37,359,009","9,078",424.34,10.31%,"3,852,199",116.56,"1,085,184",28.17%,17.24%,"$3,430,203,620.67"
1,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10100,"Aberdeen, SD","7,526",3,231,9.21%,693,3,174,25.11%,18.73%,"$588,150.97"
2,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10140,"Aberdeen, WA","17,349",18,110.44,11.46%,"1,988",18,675,33.95%,19.55%,"$1,786,700.58"
3,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10180,"Abilene, TX","25,198",11,271.09,11.83%,"2,982",11,907,30.42%,19.76%,"$2,003,346.18"
4,2015-01-01 to 2015-12-31,Ambulance (Emergency & Non-Emergency),CBSA,10220,"Ada, OK","7,570",5,104.8,6.92%,524,5,183,34.92%,12.12%,"$359,960.14"


<table style="border:1px solid black;">
  <tr>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Quantitative (interval)</th>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Quantitative (ratio)   </th>
  </tr>
  <tr>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">reference_period</td>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">number_of_fee_for_service_beneficiaries,
number_of_providers,
average_number_of_users_per_provider,
percentage_of_users_out_of_ffs_beneficiaries,
number_of_users,
average_number_of_providers_per_cbsa,
number_of_dual_eligible_users,
percentage_of_dual_eligible_users_out_of_total_users,
percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries,
total_payment
</td>
  </tr>
</table>

<table>
  <tr>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Qualitative (nominal)</th>
    <th width="200px" style="background-color: lightgrey; border:1px solid black;">Qualitative (ordinal)</th>
  </tr>
  <tr>
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">type_of_service,
cbsa,
cbsatitle
    <td style="background-color: lightblue; vertical-align: top; border:1px solid black;">
aggregation_level
    </td>
  </tr>
</table>

#### 2. Remove Punctuations

In [ ]:
# remove percent and dollar signs
cbsa = cbsa.replace('%', '', regex=True)
cbsa = cbsa.replace('\$', '', regex=True)

# remove aggregate
cbsa = cbsa[cbsa['cbsa'] != '--ALL--']

In [ ]:
# split reference period to start and end dates
cbsa[['city', 'state']] = cbsa['cbsatitle'].str.split(', ', expand=True)
cbsa.insert(0, 'state', cbsa.pop('state'))
cbsa.insert(0, 'city', cbsa.pop('city'))
# cbsa.drop(["cbsatitle"], axis = 1, inplace= True)

In [ ]:
# split reference period to start and end dates
cbsa[['start_date', 'end_date']] = cbsa['reference_period'].str.split(' to ', expand=True)
cbsa.insert(0, 'end_date', cbsa.pop('end_date'))
cbsa.insert(0, 'start_date', cbsa.pop('start_date'))
cbsa.drop(["reference_period"], axis = 1, inplace= True)

#### 3. Convert Attributes

In [ ]:
# convert date columns to datetime
cbsa[['start_date', 'end_date']] = cbsa[['start_date', 'end_date']].apply(pd.to_datetime)

In [ ]:
# convert numerical variables to integer or float
convert_numerical = [
    'number_of_fee_for_service_beneficiaries',
    'number_of_providers',
    'number_of_users',
    'number_of_dual_eligible_users',
    'average_number_of_users_per_provider',
    'percentage_of_users_out_of_ffs_beneficiaries',
    'average_number_of_providers_per_cbsa',
    'percentage_of_dual_eligible_users_out_of_total_users',
    'percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries',
    'total_payment'
]

for column in convert_numerical:
    cbsa[column] = cbsa[column].str.replace(' ', '').str.replace(',', '')
    cbsa[column] = pd.to_numeric(cbsa[column], downcast=None)

In [ ]:
# identify and convert to categorical
convert_to_category = [
    'type_of_service',
    'cbsa',
    'cbsatitle',
    'city',
    'state',
    'aggregation_level',
]
cbsa[convert_to_category] = cbsa[convert_to_category].astype('category')

In [ ]:
# confirm datatypes
print(cbsa.dtypes)

In [ ]:
## Dictionary
cbsa_dict={1:"Start of The selected one-year time period of observations",
2:"End of the selected one-year time period of observations",
3:"City of the CBSA",
4:"State or Territory of the CBSA",
5:"The selected health service area",
6:"A variable classifying the observation as the State, County, CBSA or Nation + Territories level",
7:"Five digit code to specify CBSA to indicate Nation + Territories level",
8:"CBSA name to indicate Nation + Territories",
9:"Total number of FFS beneficiaries in selected geographic region for specified reference period and service area",
10:"Total number of providers in selected geographic region for specified reference period and service area",
11:"The average number of users per provider in selected geographic region for specified reference period and service area",
12:"The percentage of users out of FFS beneficiaries in selected geographic region for specified reference period and service area",
13:"The total number of users in selected geographic region for specified reference period and service area",
14:"The average number of providers per CBSA for specified reference period and service area. Only populated for CBSAs",
15:"The number of users qualifying for Medicare and any level of Medicaid coverage in selected geographic region for specified reference period and service area",
16:"The percentage of dual eligible users out of the total number of users in selected geographic region for specified reference period and service area",
17:"The percentage of dual eligible users out of the total number FFS beneficiaries in selected geographic region for specified reference period and service area",
18:"The total payment in selected geographic region for specified reference period and service area"}

#### 4. Handle Null Values

In [ ]:
# check null values
cbsa.isnull().sum()

In [ ]:
# check if cbsa have sufficient users to qualify
cbsa.loc[cbsa['number_of_users'] == cbsa['number_of_users'].min(),'number_of_users']

**Observation:** The lowest number of user is 11, meaning the providers should be defined by CMS. Requirement is 10 and above. There appears to be many that are close to the cutoff. There may have been technicalities that did not qualify the providers according to CMS criteria.

**Investigate `average_number_of_users_per_provider`**

In [ ]:
# impute number providers of 0 with previous data
cbsa['number_of_providers'].replace(0, np.nan, inplace=True)

In [ ]:
# create a subset of cbsa with complete dual data
provider_subset = ['start_date','type_of_service','cbsa','number_of_providers']

cbsa_provider = cbsa[provider_subset].sort_values('start_date').dropna().drop_duplicates(
    subset=['type_of_service', 'cbsa'], keep='last')

In [ ]:
# fill missing values with the most recent available data
match_cols1 = ['cbsa','type_of_service']
cbsa = cbsa.set_index(match_cols1).fillna(cbsa_provider.set_index(match_cols1)).reset_index()

In [ ]:
# cbsa.to_csv('cbsa_providernum.csv', index=False)

In [ ]:
# due to provider circumstances 0
cbsa['number_of_providers'] = cbsa['number_of_providers'].fillna(0)

**Identify and remove outliers, under conditions that number of users are 10 but provider is 0**

**Investigate `number_of_dual_eligible_users`**

    - replace missing values with same data from the most recent previous reference period
    - estimate number of dual eligible using average qualifiers per type of sertice

In [ ]:
# create a subset of cbsa with complete dual data
dual_subset = ['start_date','type_of_service','cbsa','number_of_dual_eligible_users']

cbsa_dual = cbsa[dual_subset].sort_values('start_date').dropna().drop_duplicates(
    subset=['type_of_service', 'cbsa'], keep='last')

In [ ]:
# fill missing values with the most recent available data
match_cols2 = ['cbsa','type_of_service']
cbsa = cbsa.set_index(match_cols2).fillna(cbsa_dual.set_index(match_cols2)).reset_index()

In [ ]:
# aggregate mean of percentage_of_dual_eligible_users_out_of_total_users per service type

mean_percentage_service = cbsa.groupby('type_of_service')['percentage_of_dual_eligible_users_out_of_total_users'].mean()
mean_percentage_service = mean_percentage_service.reset_index()
mean_percentage_service

In [ ]:
match_mean_dual_service = ['type_of_service']
cbsa = cbsa.set_index(match_mean_dual_service).fillna(mean_percentage_service.set_index(match_mean_dual_service)).reset_index()

In [ ]:
# fill nulls of average_number_of_users_per_provider with num users/num providers
fill_avg_users_per_provider = cbsa['number_of_users']/cbsa['number_of_providers']
cbsa['average_number_of_users_per_provider'].fillna(fill_avg_users_per_provider, inplace=True)
cbsa['average_number_of_users_per_provider'].replace(np.inf, 0, inplace=True)

In [ ]:
# fill nulls of number_of_dual_eligible_users by average number per service type
fill_dual_users = cbsa['number_of_users']*(cbsa[
    'percentage_of_dual_eligible_users_out_of_total_users']/100)
cbsa['number_of_dual_eligible_users'].fillna(fill_dual_users, inplace=True)

In [ ]:
cbsa['percentage_of_dual_eligible_users_out_of_dual_eligible_ffs_beneficiaries'] = cbsa[
    'number_of_dual_eligible_users']/cbsa['number_of_fee_for_service_beneficiaries']

In [ ]:
# check null values
cbsa.isnull().sum()

In [ ]:
# cbsa.to_csv('cbsa_providerv5.csv', index=False)

Number of Fee-For-Service (FFS) Beneficiaries, Number of Providers, Number of Users, Number of Dual Eligible Users, Total Payment, reference_period, type_of_service, cbsa 

    - box plots of different services of ONE CBSA's independent diagnostic. measured by total_payment

In [ ]:
users_per_service = cbsa.groupby('type_of_service')['number_of_users'].sum()
users_per_service

In [ ]:
# filter all the long-term care hospitals in the dataset
long_term_subset = cbsa[cbsa['type_of_service'] =='Long-Term Care Hospitals']

# filter the top 10 with the largest total payment
top_lt = long_term_subset.nlargest(10, 'total_payment')

# how many times the same CBSA appeared in the top 10
top_cbsa_lt = top_lt['cbsatitle'].value_counts()
top_cbsa_lt

In [ ]:
cbsa['cbsatitle'].nunique()

In [ ]:
unique_states = cbsa['state'].unique()
unique_states
# for state in unique_states:
#    print (state)

**Observation:** some providers were 0 and therfefore error calculating `average_number_of_users_per_provider`.

Number of Fee-For-Service (FFS) Beneficiaries, Number of Providers, Number of Users, Number of Dual Eligible Users, Total Payment, reference_period, type_of_service, cbsa

All service types will be aggregated, box plot, compared by payment. Isolate and examine which has the most outliers.

#### 5. Handle `total_payment` to ensure  comparability across CBSA

**Apply Cost of Health Index to `total_payment`**

In [ ]:
# cost of health index dictionary
cost_index = {
    'AK' : 1.48,
    'AL' : 0.87,
    'AL-GA' : 0.87,
    'AR' : 0.88,
    'AR-MO' : 0.88,
    'AR-OK' : 0.88,
    'AZ' : 0.94,
    'CA' : 1.09,
    'CO' : 1.00,
    'CT' : 1.09,
    'DC-VA-MD-WV' : 1.47,
    'DE' : 1.03,
    'FL' : 0.97,
    'GA' : 1.00,
    'GA-AL' : 1.00,
    'GA-SC' : 0.91,
    'HI' : 1.23,
    'IA' : 0.99,
    'IA-IL' : 0.99,
    'IA-IL-MO' : 0.99,
    'IA-NE-SD' : 0.99,
    'ID' : 0.99,
    'ID-WA' : 0.99,
    'IL' : 0.95,
    'IL-IN-WI' : 0.95,
    'IL-MO' : 0.95,
    'IN' : 0.96,
    'IN-KY' : 0.96,
    'IN-MI' : 0.96,
    'KS' : 0.97,
    'KY' : 0.84,
    'KY-IL' : 0.84,
    'KY-IN' : 0.84,
    'LA' : 0.99,
    'MA' : 1.13,
    'MA-CT' : 1.13,
    'MA-NH' : 1.13,
    'MD' : 0.97,
    'MD-DE' : 0.97,
    'MD-WV' : 0.97,
    'ME' : 1.03,
    'MI' : 0.93,
    'MI-WI' : 0.93,
    'MN' : 1.09,
    'MN-WI' : 1.09,
    'MO' : 0.90,
    'MO-IL' : 0.90,
    'MO-KS' : 0.90,
    'MS' : 0.98,
    'MS-LA' : 0.98,
    'MT' : 0.98,
    'NC' : 1.08,
    'NC-SC' : 1.08,
    'ND' : 1.11,
    'ND-MN' : 1.11,
    'NE' : 0.98,
    'NE-IA' : 0.98,
    'NH' : 1.13,
    'NH-VT' : 1.13,
    'NJ' : 0.99,
    'NM' : 0.99,
    'NV' : 0.92,
    'NY' : 1.06,
    'NY-NJ-PA' : 1.06,
    'OH' : 0.98,
    'OH-KY-IN' : 0.98,
    'OH-PA' : 0.98,
    'OK' : 0.93,
    'OR' : 1.05,
    'OR-ID' : 1.05,
    'OR-WA' : 1.05,
    'PA' : 0.94,
    'PA-NJ' : 0.96,
    'PA-NJ-DE-MD' : 0.94,
    'PR' : 0.69,
    'RI-MA' : 1.08,
    'SC' : 0.95,
    'SC-NC' : 0.95,
    'SD' : 0.98,
    'TN' : 0.88,
    'TN-GA' : 0.88,
    'TN-KY' : 0.88,
    'TN-MS-AR' : 0.88,
    'TN-VA' : 0.88,
    'TX' : 0.94,
    'TX-AR' : 0.94,
    'UT' : 0.91,
    'UT-ID' : 0.91,
    'VA' : 1.02,
    'VA-NC' : 1.02,
    'VA-WV' : 1.02,
    'VT' : 1.06,
    'WA' : 1.20,
    'WI' : 1.12,
    'WI-MI' : 1.12,
    'WI-MN' : 1.12,
    'WV' : 0.97,
    'WV-KY-OH' : 0.97,
    'WV-OH' : 0.97,
    'WV-VA' : 0.97,
    'WY' : 1.00,
    'WY-ID' : 1.00
}

In [ ]:
# use dictionary to map states to their cost of health index
cbsa['cost_of_health_index'] = cbsa['state'].map(cost_index)
cbsa['total_payment_adjusted'] = cbsa['total_payment']*cbsa['cost_of_health_index']

**Normalize `total_payment_adjusted` by health service utilization**

In [ ]:
# create column for normalized total payment
cbsa['total_payment_adj_z'] = cbsa['total_payment_adjusted']/cbsa['number_of_users']

In [ ]:
# statistics
cbsa.describe()

In [ ]:
# visualization to see the total payment across different service types
service_payment = cbsa.groupby('type_of_service', as_index=False)['total_payment_adjusted'].sum()
service_payment.sort_values(by = ['total_payment_adjusted'], inplace=True)
fig = plt.figure(figsize = (8,15))
plt.barh(service_payment['type_of_service'], service_payment['total_payment_adjusted'])
plt.title("Total Payment for Healthcare Services by Type of Service")
plt.xlabel("Sum of Total Payments")
plt.ylabel("Service Types")

# format x-axis for readability
def format_dollars(x, pos):
    return '${:,.0f}B'.format(x/1e9)
    
plt.gca().xaxis.set_major_formatter(ticker.FuncFormatter(format_dollars))
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x='type_of_service', y='total_payment_adj_z', data= cbsa)
plt.title('Average Payment per User Across Service Types')
plt.xlabel('Type of Services')
plt.ylabel('Average Payment per User')
plt.xticks(rotation=90)

# format y-axis for readability
def format_dollars(y, pos):
    return '${:,.0f}'.format(y)
    
plt.gca().yaxis.set_major_formatter(ticker.FuncFormatter(format_dollars))


plt.show()

#### 6. Establish Metrics

#### 7. Standardize Remaining Numeric Values

#### 8. Identify and Examine Outliers

## Determine Data Mining Task


    1. Analyze and find correlation between features
    2. Build and test machine learning models
    3. Select best performing model
    4. Feature importance
    5. Deployment Planning

## Choose Data Mining Methods

## Apply Methods and Select Final Model

## Evaluate Performance